<a href="https://colab.research.google.com/github/ericatsu/Machine-Learning/blob/main/trail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import numpy as np
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

base_dir = "/content/drive/MyDrive/msc_capsmodel/dataset"
test_dir = os.path.join(base_dir, 'testing')
train_dir = os.path.join(base_dir, 'training')

Mounted at /content/drive


In [ ]:
# Define constants
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 32
NUM_CLASSES = 4
EPOCHS = 50

# Data loading and preprocessing with augmentation
def load_data(data_dir, is_training=True):
    if is_training:
        datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            zoom_range=0.2,
            brightness_range=[0.8, 1.2],
        )
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

train_data = load_data(train_dir, is_training=True)
test_data = load_data(test_dir, is_training=False)

Found 5711 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:
# Inspect a batch of data
for data_batch, labels_batch in train_data:
    print(data_batch.shape)
    print(labels_batch.shape)
    break

(32, 128, 128, 3)
(32, 4)


In [ ]:
# Squash activation function for capsules
def squash(vectors, axis=-1):
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis=axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.sqrt(s_squared_norm + 1e-8)
    return scale * vectors

In [ ]:
# Capsule layer
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings

    def build(self, input_shape):
        self.W = self.add_weight(shape=[1, input_shape[1], self.num_capsules, self.dim_capsules, input_shape[-1]],
                                 initializer='glorot_uniform',
                                 name='W')

    def call(self, inputs):
        u = tf.expand_dims(inputs, 2)
        u = tf.expand_dims(u, 3)
        u_hat = tf.reduce_sum(self.W * u, axis=-1)

        b = tf.zeros(shape=[tf.shape(inputs)[0], inputs.shape[1], self.num_capsules, 1])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            s = tf.reduce_sum(c * u_hat, axis=1, keepdims=True)
            v = squash(s, axis=-1)
            if i < self.routings - 1:
                b += tf.reduce_sum(u_hat * v, axis=-1, keepdims=True)

        return tf.squeeze(v, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsules)

In [ ]:
# Custom loss function for capsule network
def margin_loss(y_true, y_pred):
    num_classes = tf.shape(y_pred)[-1]
    y_true = tf.one_hot(tf.argmax(y_true, axis=1), depth=num_classes)
    m_plus = 0.9
    m_minus = 0.1
    lambda_val = 0.5

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.0 - 1e-9)

    L = y_true * tf.square(tf.maximum(0., m_plus - y_pred)) + \
        lambda_val * (1 - y_true) * tf.square(tf.maximum(0., y_pred - m_minus))

    return tf.reduce_mean(tf.reduce_sum(L, axis=1))

# Create the hybrid CNN-Capsule model
def create_hybrid_model():
    # Load pre-trained ResNet50
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom layers on top
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = tf.keras.layers.Reshape((-1, 256))(x)
    capsule = CapsuleLayer(num_capsules=NUM_CLASSES, dim_capsules=16)(x)
    outputs = tf.keras.layers.Lambda(lambda z: tf.sqrt(tf.reduce_sum(tf.square(z), axis=-1)))(capsule)
    model = Model(inputs=base_model.input, outputs=outputs)

    return model

In [ ]:
# Create and compile the model
model = create_hybrid_model()
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss=margin_loss, metrics=['accuracy'])

# Print model summary
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 134, 134, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 64, 64, 64)     │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 64, 64, 64)     │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 64, 64, 64)     │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 66, 66, 64)     │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 32, 32, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 32, 32, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 32, 32, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 32, 32, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 32, 32, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 32, 32, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 32, 32, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 24,128,640 (92.04 MB)

 Trainable params: 540,928 (2.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Learning rate scheduler
def lr_schedule(epoch):
    lr = 1e-4
    if epoch > 10:
        lr *= 0.1
    if epoch > 20:
        lr *= 0.1
    return lr

In [ ]:
# Train the model
history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=test_data,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_schedule)]
)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


179/179 ━━━━━━━━━━━━━━━━━━━━ 3728s 20s/step - accuracy: 0.2976 - loss: 0.4961 - val_accuracy: 0.3905 - val_loss: 0.3356 - learning_rate: 1.0000e-04
Epoch 2/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 65s 354ms/step - accuracy: 0.4008 - loss: 0.3381 - val_accuracy: 0.4935 - val_loss: 0.3157 - learning_rate: 1.0000e-04
Epoch 3/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 77s 327ms/step - accuracy: 0.4519 - loss: 0.3240 - val_accuracy: 0.4584 - val_loss: 0.3337 - learning_rate: 1.0000e-04
Epoch 4/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 61s 330ms/step - accuracy: 0.4743 - loss: 0.3158 - val_accuracy: 0.4584 - val_loss: 0.3180 - learning_rate: 1.0000e-04
Epoch 5/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 83s 332ms/step - accuracy: 0.4768 - loss: 0.3132 - val_accuracy: 0.4439 - val_loss: 0.3225 - learning_rate: 1.0000e-04
Epoch 6/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 60s 328ms/step - accuracy: 0.4874 - loss: 0.3052 - val_accuracy: 0.4912 - val_loss: 0.3242 - learning_rate: 1.0000e-04
Epoch 7/50
179/179 ━━━━━━━━━━━━━━━━━━━━ 81s 324ms/step - 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test accuracy: {test_accuracy:.4f}")